In [1]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

mlp_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlp_model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2551 - accuracy: 0.9269
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1097 - accuracy: 0.9674
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0739 - accuracy: 0.9776
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0561 - accuracy: 0.9824
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0432 - accuracy: 0.9867


In [2]:
# Keras 모델 변환
converter = tf.lite.TFLiteConverter.from_keras_model(mlp_model)
tflite_model = converter.convert()

In [3]:
# 변환한 모델 tflite로 저장
with open('./keras_model.tflite', 'wb') as f:
  f.write(tflite_model)

In [4]:
# SavedModel 저장
tf.saved_model.save(mlp_model, "./mlp_model/")
# 또는
mlp_model.save("./mlp_model/")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./mlp_model/assets
INFO:tensorflow:Assets written to: ./mlp_model/assets
INFO:tensorflow:Assets written to: ./mlp_model/assets
INFO:tensorflow:Assets written to: ./mlp_model/assets


In [6]:
# HDF5 저장
mlp_model.save("./mlp_model.h5")

In [5]:
# SavedModel 불러오기
saved_model = tf.keras.models.load_model("./mlp_model/")

In [7]:
# # HDF5 불러오기
h5_model = tf.keras.models.load_model("./mlp_model.h5")

In [8]:
# SavedModel 변환
converter = tf.lite.TFLiteConverter.from_saved_model("./mlp_model/")
tflite_model = converter.convert()

In [9]:
with open('./saved_model.tflite', 'wb') as f:
  f.write(tflite_model)

In [10]:
class Inc(tf.keras.layers.Layer):
    def call(self, inputs):
        return inputs + 1
inc = Inc()


In [ ]:
class Inc_Graph(tf.keras.layers.Layer):
    @tf.function
    def call(self, inputs):
        return inputs + 1
inc_g = Inc_Graph()

In [11]:
inc_g2 = tf.function(inc)


In [12]:
print(inc_g(tf.constant(3)))
print(inc_g(tf.constant([3, 2])))
print(inc_g(tf.constant([[3, 2], [1.0, 5.0]])))

print(inc_g2(tf.constant(3)))
print(inc_g2(tf.constant([3, 2])))
print(inc_g2(tf.constant([[3, 2], [1.0, 5.0]])))

tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([4 3], shape=(2,), dtype=int32)
tf.Tensor(
[[4. 3.]
 [2. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([4 3], shape=(2,), dtype=int32)
tf.Tensor(
[[4. 3.]
 [2. 6.]], shape=(2, 2), dtype=float32)


In [13]:
print(inc_g(tf.constant(4)))
print(inc_g2(tf.constant(4)))

tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)


In [14]:
concrete_fun = inc_g.call.get_concrete_function(tf.TensorSpec(shape=(1, 3), dtype=tf.float32))
print(concrete_fun(tf.constant([[1.0, 2.0, 3.0]])))

concrete_fun = inc_g2.get_concrete_function(tf.TensorSpec(shape=(1, 3), dtype=tf.float32))
print(concrete_fun(tf.constant([[1.0, 2.0, 3.0]])))

tf.Tensor([[2. 3. 4.]], shape=(1, 3), dtype=float32)
tf.Tensor([[2. 3. 4.]], shape=(1, 3), dtype=float32)


In [16]:
mlp_model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

graph_model = tf.function(mlp_model)
concrete_func = graph_model.get_concrete_function(
  tf.TensorSpec(shape=mlp_model.inputs[0].shape, dtype=mlp_model.inputs[0].dtype))


In [ ]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

with open('./concrete_func_model.tflite', 'wb') as f:
  f.write(tflite_model)

In [15]:
class MLP_Model(tf.keras.Model):
    def __init__(self):
        super(MLP_Model, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(128, activation='relu')
        self.softmax = tf.keras.layers.Dense(10, activation='softmax')
    
    @tf.function
    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense(x)
        return self.softmax(x)

mlp_model = MLP_Model()
concrete_func = mlp_model.call.get_concrete_function(
    tf.TensorSpec(shape=(None, 28, 28), dtype=tf.float32))


In [ ]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

with open('./concrete_func_model.tflite', 'wb') as f:
  f.write(tflite_model)